## 1. Original Try

In [1]:
# from langchain.document_loaders import PyPDFLoader
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.text_splitter import CharacterTextSplitter
# from dotenv import load_dotenv
# import openai
# import os
# import pypdf
# import faiss
# from PyPDF2 import PdfReader

### Creating the Vector store by loading in documents

In [2]:
#create the persistant vector store.
#make sure app works with new API key
#rework the app to retrieve the vector store
#rework app to make sure that the formating is correct and the chat is working correctly
#Buy a URL on Route 53. 
#figure out how to host it.

In [3]:
#import PDF documents
directory = 'documents'
text = ""
 
# iterate over files directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        pdf_reader = PdfReader(f)
        for page in pdf_reader.pages:
            text += page.extract_text()

In [4]:
#Chunk text
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size = 700,
    chunk_overlap = 200,
    length_function = len
)

docs = text_splitter.split_text(text)

In [5]:
docs

['UCWDC® Rules,  \nContest Procedures  \nand Scoring Format  \n2023 - 2025 \nCOUPLES  \n \nUNITED COUNTRY WESTERN DANCE COUNCIL\uf0d2  \nCOPYRIGHT 1987,  REVISED 2023 Table of Contents  \nI. GENERAL INFORMATION  ................................ ................................ ................................ ..........................  1 \nA. UNITED COUNTRY WESTERN DANCE COUNCIL ® ................................ ................................ ................................ ................................  1 \nB. UCWDC’ S MISSION AND VISION  ................................ ................................ ................................ ................................ ..................  1',
 'B. UCWDC’ S MISSION AND VISION  ................................ ................................ ................................ ................................ ..................  1 \nC. UCWDC  BUSINESS OFFICE  ................................ ................................ .......

In [6]:
#Create embedings 
embeddings = OpenAIEmbeddings(model="text-search-ada-doc-001")

#Use Langchain to create the embeddings using text-embedding-ada-002
db = FAISS.from_texts(texts=docs,embedding=embeddings)

#save the embeddings into FAISS vector store
db.save_local("FAISS")

In [9]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [8]:
def get_conversation_chain(vectorstore):
    llm = ChatOpenAI()
    # llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

In [9]:
#get vector store
embedding_function = OpenAIEmbeddings(model="text-search-ada-doc-001")
vectorstore = FAISS.load_local("FAISS",embedding_function)


In [18]:
conversation = get_conversation_chain(db)

In [19]:
conversation

ConversationalRetrievalChain(memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history'), callbacks=None, callback_manager=None, verbose=False, combine_docs_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=ChatPromptTemplate(input_variables=['question', 'context'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', val

In [11]:
print(conversation)

memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history') callbacks=None callback_manager=None verbose=False combine_docs_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, prompt=ChatPromptTemplate(input_variables=['question', 'context'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True), additional_

In [12]:
def handle_userinput(user_question):
    response = conversation({'question': user_question})
    chat_history = response['chat_history']
    return chat_history

#### Testing first try

In [26]:
handle_userinput("What makes a qualified win")

[HumanMessage(content='what is rise and fall', additional_kwargs={}, example=False),
 AIMessage(content='Rise and Fall is a style of movement in partner dancing, particularly in ballroom dancing. It refers to the up and down motion of the dancers as they move across the dance floor. During the "rise" portion, the dancers elevate and create a feeling of lightness, while during the "fall" portion, they lower and create a grounded and stable connection with the floor. This technique is used to create a smooth and flowing movement in dances such as waltz and foxtrot.', additional_kwargs={}, example=False),
 HumanMessage(content='what is superstars', additional_kwargs={}, example=False),
 AIMessage(content='There is no mention of "superstars" in the provided context. Therefore, I do not have the information to provide a definition.', additional_kwargs={}, example=False),
 HumanMessage(content='what is line dance', additional_kwargs={}, example=False),
 AIMessage(content='Line dance refers t

In [114]:
import os

USER = os.getenv('OPENAI_API_KEY')

In [115]:
print(USER)

In [14]:
handle_userinput("What is chacha")

[HumanMessage(content='what is Superstars line dance?', additional_kwargs={}, example=False),
 AIMessage(content='Based on the provided context, there is no specific information about a line dance called "Superstars." Therefore, it is not possible to determine what Superstars line dance is.', additional_kwargs={}, example=False),
 HumanMessage(content='What is chacha', additional_kwargs={}, example=False),
 AIMessage(content="I don't have enough information to provide a definition of chacha based on the given context.", additional_kwargs={}, example=False)]

# New Try

### Creating the Vector store and load in the documents

In [7]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
import os
from langchain.document_loaders import PyPDFLoader

In [8]:
#import PDF documents
directory = 'documents'
raw_docs = []
 
# iterate over files directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        loader = PyPDFLoader(f)
        raw_docs.extend(loader.load())

In [9]:
#previewing the documents in the list
raw_docs

[Document(page_content=' \n \n  UCWDC® Rules,  \nContest Procedures  \nand Scoring Format  \n2023 - 2025 \nCOUPLES  \n \nUNITED COUNTRY WESTERN DANCE COUNCIL\uf0d2  \nCOPYRIGHT 1987,  REVISED 2023 ', metadata={'source': 'documents\\UCWDC Rules_Couples_VERIFIED_FINAL V2.pdf', 'page': 0}),
 Document(page_content='Table of Contents  \nI. GENERAL INFORMATION  ................................ ................................ ................................ ..........................  1 \nA. UNITED COUNTRY WESTERN DANCE COUNCIL ® ................................ ................................ ................................ ................................  1 \nB. UCWDC’ S MISSION AND VISION  ................................ ................................ ................................ ................................ ..................  1 \nC. UCWDC  BUSINESS OFFICE  ................................ ................................ ................................ ..................

In [10]:
# Load the document, split it into chunks, embed each chunk and load it into the vector store.
#raw_documents = TextLoader(r"documents\UCWDC Rules_Line Dance.pdf").load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = text_splitter.split_documents(raw_docs)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_c863ee10a451b2c674f184822cd1313f in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_c863ee10a451b2c674f184822cd1313f in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_

In [6]:
db.save_local("FAISS_Country")

### Testing it with some code

In [7]:
from langchain.memory import ConversationSummaryMemory

In [10]:
llm = ChatOpenAI()

In [11]:
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)

In [12]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
retriever = db.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [14]:
qa("what is waltz")

{'question': 'what is waltz',
 'chat_history': [SystemMessage(content='', additional_kwargs={})],
 'answer': 'The waltz is a smooth, progressive dance that uses patterns that move diagonally and are performed with rise and fall, accentuated with swing and sway, emphasizing hover. The basic timing for the waltz is 1 2 3 4 5 6.'}

In [15]:
def handle_userinput1(user_question):
    response = qa({"question": user_question})
    chat_history = response['answer']
    return chat_history

In [16]:
qa("How many line dance categories are there")

{'question': 'How many line dance categories are there',
 'chat_history': [SystemMessage(content='The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\n\nThe human asks the AI about the waltz. The AI explains that the waltz is a smooth, progressive dance that uses specific patterns and timing.', additional_kwargs={})],
 'answer': 'There are six categories for line dances: Rise & Fall, Pulse, Smooth, Cuban, Funky/Street, and Novelty/Stage.'}

In [17]:
handle_userinput1("are arials allowed?")

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_060da41ed4718893ccd34469f7ecaaf5 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_060da41ed4718893ccd34469f7ecaaf5 in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_

'No, aerials are not allowed in line dances according to the UCWDC Dance Rules.'

In [80]:
query = "Are arials allowed"
docs = db.similarity_search(query)
print(docs[1].page_content)

- 
UCWDC Dance Rules, Contest Procedures and Scoring Format – LINE DANCE              Page 6 of 16 c. Movement allowances and limitations for Line Dance Showcase and Solo Medleys within SuperStars, 
RisingStars , and Crown : 
i. A competitor  may begin the dance facing any direction of his or her choosing and may begin 
the dance at any time . 
ii. In Line Dance Showcase, a competitor  must dance repetitive patter ns according to the 
phrasing of the music.  
iii. Aerials are not allowed in Crown  Solo Medley  
iv. There are no movement limitations in SuperStars and RisingStars  Solo Medley . 
 
H. Floor Craft  
1. For all dances, competitor s shall  always be courteous on the floor , as well as vigilant so as not to interfere 
with other competitor s. 
2. For Line Dance categories, once the competitor s evenly space or stagger themselves on the floor relative to 
each other, everyone’s “dance space” is revealed. Competitor s should perform to  reasonable maintain their 
relative posit